In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.geoclaw import dtopotools
from scipy.interpolate import interp1d

In [ ]:
sys.path.insert(0,'../../new_python')
import topotools, marching_front

## Define ocean topography

This simple topography is piecewise linear in $x$ (longitude) with a continental shelf and beach, and constant in the $y$ (latitude) direction.  It is placed at the equator.

In [ ]:
# Define piecewise linear function (unequally spaced):
xocean = array([-2,-1,-0.5,-0.1,0.1])
zocean = array([-3000,-3000,-100,-100,100])

# Interpolate to equally spaced grid for topofile:
xo = arange(-2,0.2,0.1)
yo = array([-2,2])
zfunc = interp1d(xocean,zocean,fill_value="extrapolate")
zo = zfunc(xo)

# Convert to 2d arrays:
Xo,Yo = meshgrid(xo,yo)
Zo = vstack((zo,zo))

In [ ]:
figure(figsize=(12,5))
subplot(121)
contourf(Xo,Yo,Zo)
colorbar()
title('Ocean Topography')
subplot(122)
plot(xo,zo,'k-')
fill_between(xo,zo,maximum(zo,0),color=[.5,.5,1])
title('Topography on transect')

### Save as a topofile:

In [ ]:
topo = topotools.Topography()
topo.set_xyZ(xo,yo,Zo)

topo.write('input_files/topo_ocean.tt3', topo_type=3, Z_format="%11.3e")

## Define topo for small coastal region

We define some more complicated topography on a finer grid over a small coastal region with 1/3 arcsecond resolution, chosen to be aligned with integer multiples of degrees (e.g. a grid point at longitude `x=0` and latitude `y=0`) as typical of real DEMs from NCEI.  This is important when aligning fgmax grids in `setrun.py`.   

We will use a cutoff function so that this fine-scale topo matches the linear beach profile of the ocean topography along the edges of this rectangle.  The cutoff is 1 in the center of the rectangle and decays to 0 at the edges:

In [ ]:
# choose DEM grid points:
arcsec13 = 1./(3*3600.)  # 1/3 arcsecond
print('arcsec13 = %.6f degrees = %.2f meters' % (arcsec13,arcsec13*111e3))
x = arange(-30*arcsec13, 85*arcsec13, arcsec13)
y = arange(-55*arcsec13, 55*arcsec13, arcsec13)
X,Y = meshgrid(x,y)
print('X.shape = ', X.shape)

x1,x2 = x.min(), x.max()
y1,y2 = y.min(), y.max()

# define and plot the cutoff function:

cutoff = 1. / (1. + exp(1e4*(X-0.007)) + exp(1e4*(-0.002-X)) \
                  + exp(1e4*(Y-0.004)) + exp(1e4*(-0.004-Y)))

contourf(X,Y,cutoff)
colorbar()
title('Cutoff function');

The topography in this region is the linearly sloping beach augmented by two Gaussian dips:

In [ ]:
Z0 = 1e3*X  # sloping beach with slope 1 km / degree which is roughly 1/100.
R1 = (X-0.004)**2 + (Y-0.002)**2
Z = -9*exp(-500000*R1)
R2 = (X-0.005)**2 + (Y+0.002)**2
Z += -4*exp(-100000*R2)
Z *= cutoff
Z += Z0

### Plot the coastal topography:

In [ ]:
# colors:
c = [[.2,.2,1],[.5,.5,1],[.8,.8,1],[.8,1,.8],[.2,.8,0],[.9,.8,.2]]

figure(figsize=(12,6))
subplot(121)
contourf(X,Y,Z,[-2,-1,0,1,2],colors=c,extend='both')
cb = colorbar(shrink=0.7)
cb.set_label('meters')
contour(X,Y,Z,[-2,-1,0,1,2],colors=['b','b','r','g','g'])
gca().set_aspect(1.)
xticks(rotation=20)
xlabel('Longitude')
ylabel('Latitude')

subplot(122)
contourf(X*111e3,Y*111e3,Z,[-2,-1,0,1,2],colors=c,extend='both')
cb = colorbar(shrink=0.7)
cb.set_label('meters')
contour(X*111e3,Y*111e3,Z,[-2,-1,0,1,2],colors=['b','b','r','g','g'])
gca().set_aspect(1.)
xticks(rotation=20)
xlabel('meters');

The plot on the right above shows the same topography as on the left, but with x,y units of meters to better show the scale.  Recall that 1 degree is about 111 km and 1/3 arcsec is about 10 meters.

In the plots above, the red contour is at $Z = 0$, and hence is the "shoreline".  However, the isolated "lake" with elevation $Z < 0$ could be dry land below sea level.  Normally with GeoClaw this region would be filled with water initially up to $Z = 0$ everywhere.  Below we discuss how to force this region to be initialized as dry if it is in fact dry land.

### Save this as a topofile:

In [ ]:
topo = topotools.Topography()
topo.set_xyZ(x,y,Z)

topo.write('input_files/topo_shore.tt3', topo_type=3, Z_format="%11.3e")
topo.plot()

### Plot both topo sets together

The coastal region above is very small compared to the ocean region defined above.  Here we plot both together:

In [ ]:
def plot_topo(add_colorbar=False):
    contourf(Xo,Yo,Zo,[-2,-1,0,1,2],colors=c,extend='both')
    contourf(X,Y,Z,[-2,-1,0,1,2],colors=c,extend='both')
    if add_colorbar: 
        cb = colorbar()
        cb.set_label('meters')
    contour(X,Y,Z,[-2,-1,0,1,2],colors=['b','b','r','g','g'])
    plot([x1,x1,x2,x2,x1],[y1,y2,y2,y1,y1],'k')
    gca().set_aspect(1.)
    xticks(rotation=20)
    xlabel('Longitude')
    ylabel('Latitude')

figure(figsize=(12,6))
subplot(121)
plot_topo()
title('Ocean Topography')
subplot(122)
plot_topo(add_colorbar=True)
axis([-0.004, 0.01, -0.01, 0.01])
title('Zoom around shore');

In the plot on the left above, the black rectangle showing the extent of the coastal DEM is barely visible.  Zooming in shows that the topography does match up near the edges of this rectangle.  In GeoClaw the finest available topography is used when computing cell-averaged topo values, so the coastal DEM will be used for any cell that overlaps this region. 

## Create dtopo for an earthquake source:

We define a simple earthquake in which there is uniform slip on a single subfault of dimension 100 km by 50 km, with parameters chosen to be reasonable for a subduction zone event offshore:

In [ ]:
subfault = dtopotools.SubFault()
subfault.strike = 0.
subfault.length = 50.e3
subfault.width = 100.e3
subfault.depth = 10.e3
subfault.slip = 5.
subfault.rake = 90.
subfault.dip = 10.
subfault.longitude = -1.
subfault.latitude = 0.
subfault.coordinate_specification = "top center"

fault = dtopotools.Fault()
fault.subfaults = [subfault]

print("Earthquake magnitude: Mw = %.2f" % fault.Mw())
dtopo_fname = 'input_files/dtopo_test.tt3'
print("Using Okada model to create dtopo file", dtopo_fname)

x = linspace(-2, 1, 100)
y = linspace(-1, 1, 100)
times = [1.]

fault.create_dtopography(x,y,times)
dtopo = fault.dtopo

dtopo.write(dtopo_fname, dtopo_type=3)

figure(figsize=(12,6))
ax = subplot(121)
dtopo.plot_dZ_colors(2.,axes=ax,dZ_interval=0.5)
contour(Xo,Yo,Zo,[-110,-90,0],colors=['b','b','r'],linestyles='--')
ax.set_aspect(1.)
axis([-2,0.5,-2,2])
xlabel('Longitude')
ylabel('Latitude')

ax = subplot(122)
ylat = 0.
jlat = where(dtopo.y<=ylat)[0].max()
plot(dtopo.x, dtopo.dZ[0,jlat,:],'g')
plot(dtopo.x, 0*dtopo.x, 'k')
xlabel('Longitude')
title('Vertical displacement on transect at latitude %.2f' % ylat);

The left plot above shows the sea floor deformation as contours and colors, along with the extent of the continental shelf as blue dashed lines and the shoreline as a red dashed line. The plot on the right shows the vertical deformation along a transect at latitude 0 going through the coastal region of interest.  

We can compute the subsidence at the location on the shoreline where our fine scale topography is defined as:

In [ ]:
xlon = 0.
ilon = where(dtopo.x<=xlon)[0].max()
ylat = 0.
jlat = where(dtopo.y<=ylat)[0].max()
#print(ilon,jlat)
dz0 = dtopo.dZ[0,jlat,ilon]
print('Surface deformation at x=%.2f, y=%.2f is dz = %.2f meters' \
      % (xlon,ylat,dz0))

This subsidence is enough to significantly change the shoreline location, as seen below:

In [ ]:
figure(figsize=(12,6))
subplot(121)
contourf(X,Y,Z,[-2,-1,0,1,2],colors=c,extend='both')
cb = colorbar(shrink=0.7)
cb.set_label('meters')
contour(X,Y,Z,[-2,-1,0,1,2],colors=['b','b','r','g','g'])
gca().set_aspect(1.)
xticks(rotation=20)
#xlim(-0.002,0.008)
xlabel('Longitude')
ylabel('Latitude')
title('Original topo')

subplot(122)
Z_postquake = Z + dz0
contourf(X,Y,Z_postquake,[-2,-1,0,1,2],colors=c,extend='both')
cb = colorbar(shrink=0.7)
cb.set_label('meters')
contour(X,Y,Z_postquake,[-2,-1,0,1,2],colors=['b','b','r','g','g'])
gca().set_aspect(1.)
xticks(rotation=20)
#xlim(-0.002,0.008)
xlabel('Longitude')
ylabel('Latitude')
title('Subsided topo, dz = %.2f m' % dz0);

In [ ]:
Ztemp = topo.Z.copy()
Ztemp[:2,:2] = -50.

In [ ]:
wet_points = marching_front.select_by_flooding(Ztemp, Z1=-40., Z2=0., max_iters=None)

Zdry = ma.masked_array(topo.Z, wet_points)
Zwet = ma.masked_array(topo.Z, logical_not(wet_points))

In [ ]:
figure(figsize=(12,6))
contourf(X,Y,Zdry,[-2,-1,0,1,2],colors=c,extend='both')
cb = colorbar(shrink=0.7)
cb.set_label('meters')
contour(X,Y,Z,[-2,-1,0,1,2],colors=['b','b','r','g','g'])
gca().set_aspect(1.)
xticks(rotation=20)
#xlim(-0.002,0.008)
xlabel('Longitude')
ylabel('Latitude')
title('Points identified as initially dry');

In [ ]:
dry_points = 1 - wet_points
dry_points_sum = dry_points[1:-1,1:-1] + dry_points[0:-2,1:-1] + dry_points[2:,1:-1] + \
                 dry_points[1:-1,0:-2] + dry_points[0:-2,0:-2] + dry_points[2:,0:-2] + \
                 dry_points[1:-1,2:] + dry_points[0:-2,2:] + dry_points[2:,2:]
        
# initialize array to 0 everywhere:
force_dry_init = zeros(dry_points.shape)
# reset in interior to 1 if all points in the 3x3 block around it are dry:
force_dry_init[1:-1,1:-1] = where(dry_points_sum == 9, 1, 0)

In [ ]:
force_dry_init_topo = topotools.Topography()
force_dry_init_topo.set_xyZ(topo.x,topo.y,force_dry_init)

fname_force_dry_init = 'input_files/force_dry_init.data'
force_dry_init_topo.write(fname_force_dry_init, topo_type=3, Z_format='%1i')
print('Created %s' % fname_force_dry_init)